# Load Ni-Mo data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Set up the NNP and train

In [2]:
from maml.apps.pes import NNPotential

nnp = NNPotential()
nnp.train(train_structures=train_structures,
          train_energies=train_energies,
          train_forces=train_forces,
          cutoff_type=1, atom_energy={'Ni': -3.0, 'Mo': -4.14},
          r_etas = [0.5, 2.0], a_etas = [0.5, 2.0],
          r_shift=[0.0],
          zetas=[1.0, 4.0],
          r_cut=4.2, hidden_layers=[4, 4], epochs=5)

0

# Predict the energies, forces of training data

In [8]:
df_orig, df_predict = nnp.evaluate(test_structures=train_structures, 
                                   test_energies=train_energies,
                                   test_forces=train_forces, test_stresses=None)

In [9]:
df_predict[df_predict['dtype'] == 'energy']

,y_orig,n,dtype
0,-1012.786906,144.0,energy
433,-1011.509193,144.0,energy
866,-1013.797973,144.0,energy
1299,-1011.251975,144.0,energy
1732,-1011.033166,144.0,energy
2165,-1009.670699,144.0,energy
2598,-1012.086308,144.0,energy
3031,-1015.375696,144.0,energy
3464,-1010.978923,144.0,energy
3897,-1011.395658,144.0,energy


# Lattice constant, Elastic constant
### Large error due to limited training data -- 10 structures

In [ ]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [6]:
from maml.apps.pes import LatticeConstant

lc_calculator = LatticeConstant(ff_settings=nnp)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Ni Lattice a: 5.3009294107146, Lattice b: 5.3009294107146, Lattice c: 5.3009294107146


In [7]:
lc_calculator = LatticeConstant(ff_settings=nnp)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Mo Lattice a: 3.17536452382851, Lattice b: 3.17536452382851, Lattice c: 3.17536452382851


In [4]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=nnp)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Ni  C11:  245.464447449194 C12:  72.2672717252862 C44:  201.597098446191


In [5]:
Mo_ec_calculator = ElasticConstant(ff_settings=nnp)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

Mo  C11:  23914.7973436972 C12:  755.4018514041 C44:  -0.416647311230344


# Load from parameters files

In [9]:
nnp_loaded = NNPotential.from_config(input_filename='input.nn', 
                                     scaling_filename='scaling.data', 
                                     weights_filenames=['weights.028.data', 'weights.042.data'])

# Energy, force, stress prediction

In [11]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=nnp_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -1008.18727739683
forces: 
 [[ 1.11144e+00 -6.87330e-01  5.59798e-01]
 [-9.50588e-01 -6.83647e-01 -4.49679e-01]
 [-9.93645e-01 -8.44507e-02 -5.32505e-01]
 [ 1.23688e+00 -9.11643e-01 -6.67374e-02]
 [ 2.53070e-01  1.45826e-01  4.76144e-01]
 [ 1.15865e+00 -7.28244e-01 -5.05639e-01]
 [ 6.05220e-01 -5.09586e-01  1.17076e-01]
 [-1.05514e-01 -9.88972e-01 -7.27718e-02]
 [-7.39764e-01  2.09987e-01 -7.83525e-01]
 [ 2.98573e-02  4.59915e-01  1.13063e-01]
 [-5.55656e-02  6.18511e-01  5.03007e-01]
 [ 6.51934e-01 -6.74251e-02 -1.52876e-01]
 [-5.15689e-01  7.22001e-01 -4.31092e-01]
 [ 1.35868e-01  4.77962e-01  2.58576e-01]
 [ 2.85437e-01  5.66788e-01  3.59887e-01]
 [ 2.47084e-01 -1.54355e+00  4.36199e-01]
 [ 8.93904e-01 -3.56717e-01 -6.99487e-01]
 [-1.07203e+00 -8.88131e-02  6.90709e-02]
 [-7.38746e-01  3.52241e-01  9.66960e-01]
 [ 3.28816e-01  9.69687e-01 -1.24338e-01]
 [-5.98264e-01  7.69112e-02 -5.27247e-01]
 [ 6.73815e-01 -3.58007e-01 -3.03580e-01]
 [ 1.25828e+00  9.49771e-01 -1.78365e-01